In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [23]:
!pip install google-generativeai

In [24]:
import google.generativeai as genai
import os
import datetime

In [25]:
genai.configure(api_key=os.environ["GEMINI_API_KEY"])

KeyError: 'GEMINI_API_KEY'

In [35]:
logs = []

def log_event(agent, message):
    time = datetime.datetime.now().strftime("%H:%M:%S")
    logs.append(f"[{time}] [{agent}] {message}")

In [34]:
def save_report(text):
    with open("final_report.txt", "w") as f:
        f.write(text)
    return "Saved final_report.txt"

In [33]:
google_search_model = genai.GenerativeModel(
    model_name="gemini-1.5-flash",
    tools=[{"google_search": {}}]
)

def google_search_tool(query):
    log_event("SearchTool", f"Searching: {query}")
    response = google_search_model.generate_content(
        [{"google_search": {"query": query}}]
    )
    return response.text

ValueError: Unknown field for FunctionDeclaration: google_search

In [32]:
memory = []

def remember(text):
    memory.append(text)

def recall():
    return "\n".join(memory[-5:])

In [31]:
research_agent = genai.GenerativeModel("gemini-1.5-flash")

def run_research_agent(topic):
    log_event("ResearchAgent", f"Researching: {topic}")
    
    search_results = google_search_tool(topic)
    remember("Search Results: " + search_results)

    prompt = f"""
    Summarize the following search results into 8–10 key points:

    {search_results}
    """

    response = research_agent.generate_content(prompt)
    summary = response.text
    
    remember("Research Summary: " + summary)
    return summary

In [30]:
summarizer_agent = genai.GenerativeModel("gemini-1.5-pro")

def run_summarizer_agent(text):
    log_event("SummarizerAgent", "Creating detailed explanation")

    prompt = f"""
    Create a clear, detailed explanation with headings, examples, and structured points.

    TEXT:
    {text}
    """

    response = summarizer_agent.generate_content(prompt)
    summary = response.text
    
    remember("Detailed Summary: " + summary)
    return summary

In [29]:
writer_agent = genai.GenerativeModel("gemini-1.5-pro")

def run_writer_agent(topic, content):
    log_event("WriterAgent", "Writing final report")

    memory_context = recall()

    prompt = f"""
    Write a professional research report on: {topic}

    Use this content:
    {content}

    MEMORY:
    {memory_context}

    Structure:
    - Title
    - Introduction
    - Key Findings
    - Detailed Explanation
    - Conclusion
    - References
    """

    response = writer_agent.generate_content(prompt)
    final_report = response.text

    save_report(final_report)
    return final_report

In [28]:
def ai_research_pipeline(topic):
    log_event("SYSTEM", f"Starting pipeline for: {topic}")

    step1 = run_research_agent(topic)          # Agent 1
    step2 = run_summarizer_agent(step1)        # Agent 2
    final = run_writer_agent(topic, step2)     # Agent 3

    log_event("SYSTEM", "Pipeline finished")
    return final

In [27]:
topic = "Impact of Artificial Intelligence on Healthcare"
final_output = ai_research_pipeline(topic)

print(final_output)

NameError: name 'google_search_tool' is not defined

In [26]:
for log in logs:
    print(log)

[08:37:44] [SYSTEM] Starting pipeline for: Impact of Artificial Intelligence on Healthcare
[08:37:44] [ResearchAgent] Researching: Impact of Artificial Intelligence on Healthcare
[08:37:46] [SYSTEM] Starting pipeline for: Impact of Artificial Intelligence on Healthcare
[08:37:46] [ResearchAgent] Researching: Impact of Artificial Intelligence on Healthcare
